# Excercise 1: SVM

In [ ]:
%matplotlib inline 

import autograd.numpy as np
import autograd.numpy.random as npr


#from autograd import grad
import scipy.optimize

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from YourAnswer import *
from utils import *

## Data Content

We generate synthetic data ${\textbf{x}^{(i)}, y^{(i)}}_{i=1,...,n}$ for binary classification that the distribution of each class is given by:
$$
p(\mathbf{x}|y=1) \sim ~ N(\mathbf{\mu_1}, \mathbf{I})\\
p(\mathbf{x}|y=-1) \sim ~ N(\mathbf{\mu_2}, \mathbf{I})\\
where\ \mu_1 = [1 \ \   1]^T, \mu_2 = [-1 -1]^T, 
$$

In [ ]:
from sklearn.datasets import make_blobs
n_dim = 2
x_train, y_train = make_blobs(n_samples=100, n_features=n_dim, centers=[[1,1],[-1,-1]], shuffle=True, random_state=7)
x_test, y_test = make_blobs(n_samples=100, n_features=n_dim, centers=[[1,1],[-1,-1]], shuffle=True, random_state=777)
y_train = (y_train * 2) - 1
y_test = (y_test * 2) - 1

vis_data(x_train, y_train, 'r', 'Train data')
vis_data(x_test, y_test,'r', 'Test data')

## 1. Binary classification with Linear SVM

The objective function of SVM is defined as:
$$
\underset{\pmb{\theta}_1\in\mathbb{R}^d, \theta_0\in\mathbb{R}}{\operatorname{minimize}} \hspace{0.2cm} \frac{1}{2}||\pmb{\theta}_1||_2^2 + C\sum_{i=1}^nL_{Hinge}(h,(\mathbf{x}^{(i)},y^{(i)})) \\
\text{where } L_{Hinge}(h,(\mathbf{x},y)) = (1-y\cdot(\theta_1^{\top}\mathbf{x}+\theta_0))_+ = (1 - s_{\pmb{\theta}}(y,\mathbf{x}))_+ 
$$

You will implement all parts of the function step by step.

### 1.1. Score function
##### To do : Implement `score_function` in `YourAnswer.py` <br>
Score function:
$$
s_{\pmb{\theta}}(y, \mathbf{x}) = y\cdot(\pmb{\theta}_1^{\top}\mathbf{x}+\theta_0)
$$

Your score function would return following values <br>
`array([-0.88307155, -0.89749892,  0.87472908, -0.88987632, -0.89341663])` 

In [ ]:
# data for sanity check
n = 5
d = 2
C = 1
np.random.seed(1234)
x_val = np.random.randn(n,d-1)
y_val = np.random.randint(0, 2, n)
y_val[y_val==0] = -1
theta_val = np.random.randn(d)

In [ ]:
score_function(x_val, y_val, theta_val)

### 1.2. Prediction function
##### To do : Implement `prediction_function` in `YourAnswer.py` <br>

Predict function:
$$
sign(\pmb{\theta}_1^{\top}\mathbf{x}+\theta_0)
$$


Your prediction function would return following values

`array([1., 1., 1., 1., 1.])`

In [ ]:
prediction_function(x_val, theta_val)

### 1.3. Hinge loss
##### To do : Implement `hinge_loss` in `YourAnswer.py` <br>


Hinge loss:
$$
L_{Hinge}(h,(\mathbf{x},y)) = (1-s_{\pmb{\theta}}(y, \mathbf{x}))_+\\
$$

Your hinge_loss would return following values

`array([1.88307155, 1.89749892, 0.12527092, 1.88987632, 1.89341663])`

In [ ]:
hinge_loss(x_val, y_val, theta_val)

### 1.4. Objective function
##### To do : Implement `objective_function` in `YourAnswer.py` <br>

Objective function:
$$
\text{minimize}_{\pmb{\theta}_1\in\mathbb{R}^d, \theta_0\in\mathbb{R}} \frac{1}{2}||\pmb{\theta}_1||_2^2 + C\sum_{i=1}^nL_{Hinge}(h,(\mathbf{x}^{(i)},y^{(i)})) \\
$$

Your objective_function would return following values

`7.689171997714208`

In [ ]:
objective_function(theta_val, x_val, y_val, C)

### 1.5. Training Process

##### To do : Implement `update_svm` in `YourAnswer.py` <br>
In this part, you should compute the graident of objective function and update theta by batch gradient descent

In [ ]:
# initialize theta
theta_init = 1e-4 * npr.randn(n_dim+1); theta_init[-1] = 0.

In [ ]:
alpha = 5e-6
num_iters = 2500
C=1
theta_opt = update_svm(theta_init, x_train, y_train, C, num_iters, alpha)

Let’s see the obtained decision boundary and marginal hyperplanes.

In [ ]:
plot_svc(prediction_function, x_train, y_train, plot_support_vector=True, plot_mis=True, sklearn=False, score_func=score_function, w=theta_opt)

## 4. Testing

In [ ]:
plot_svc(prediction_function, x_test, y_test, plot_mis=True, sklearn=False, score_func=score_function ,w=theta_opt)

### Logisitic regression VS SVM

Let's fit the data using logistic regression first.<br>
We exploit the scikit learn model simply.

In [ ]:
logistic = LogisticRegression(C= 1.0, solver='lbfgs')
logistic.fit(x_train, y_train)

Now, we can compare results from two different methods for the given data

Because of different optimizing ways, the decision boundaries would not be exactly the same.


In [ ]:
plot_all(theta_opt, logistic, x_train, y_train)

## 5. Predicting a dataset with multimodal distribution 

In section 5, we predict a dataset with multimodal distribution by using `kernel method` in scikit-learn.

## 5.1. Multimodal dataset

In [ ]:
x_train, y_train = gen_multimodal()
x_test, y_test = gen_multimodal()

vis_data(x_train, y_train, c='r')
vis_data(x_test, y_test, c='r')

## 5.2. Kernel SVM
##### To do : Implement `gaussian_kernel` and `polynomial_kernel`  in `YourAnswer.py` <br>
Implement Gaussian kernel and polynomial kernel. <br>
This process may take a few minutes.

1. Gaussian Kernel
$$
K(\pmb{x},\pmb{z}) = \text{exp}  (-\frac{||\pmb{x}-\pmb{z}||^2 }{2\sigma^2}) \\
$$

2. Polynomial Kernel
$$
K(\pmb{x},\pmb{z}) = (\pmb{x}^T\pmb{z}+ c)^m \\
$$

##### Gaussian kernel 
$\sigma=1$ <br>

In [ ]:
_sigma=1
svm=SVC(C=1.0, kernel= gaussian_proxy_kernel(sigma=_sigma))
svm.fit(x_train, y_train)
y_predict=svm.predict(x_test)
plot_svc(svm, x_test, y_test, h=0.1, plot_support_vector=False)

##### Polynomial kernel 
m (degree)=3 and   c (bias)=1

In [ ]:
_degree=3
_bias=1

svm=SVC(C=1.0, kernel=poly_proxy_kernel(degree=_degree,bias=_bias))
svm.fit(x_train, y_train)
y_predict=svm.predict(x_test)
plot_svc(svm, x_test, y_test, h=0.1, plot_support_vector=False)

##### Check your implementation. 

Gaussian kernel <br>
![nn](./gaussian_figure.png) <br>
Poly kernel <br>
![nn](./poly_figure.png)

Let's find the best parameters by grid search. 
##### To do : Implement `gridsearch`  in `YourAnswer.py` <br>

In [ ]:
# Set the parameters for cross-validation
# You can change parameters to find optimal parameter(C, gamma)
# and also modify another parameter in GridSearchCV(CV).

tuned_parameters = [{'C': [0.01, 0.1, 1, 10, 100],
                     'gamma': [0.5, 1,2,3,4]}]

clf = gridsearch(tuned_parameters, x_train, y_train)

print("Grid scores")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

In [ ]:
clf.best_params_

In [ ]:
# Check the accuracy of the best model
clf.best_estimator_.score(x_test,y_test)

For comparson of separting hyperplane, we will train the linear-SVM by the fuction you implemented in the multimodal dataset.


In [ ]:
alpha = 1e-5
num_iters = 1000
C=1
theta_opt = update_svm(theta_init, x_train, y_train, C, num_iters, alpha,print_log=False)

Let's compare the separating hyperplane of Kernel-SVM to that of Linear-SVM in the multimodal dataset.<br>
You can see that the kernel SVM can make a nonlinear decision boundary. Thus, when the dataset is hard to separate  <br>
linearly, e.g., the multimodal dataset, Kernel-SVM is a more reasonable choice than the Linear-SVM. 

In [ ]:
plot_svc_with(clf.best_estimator_, x_test, y_test, plot_support_vector=False,w =theta_opt)